In [1]:
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt


In [2]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, LancasterStemmer, PorterStemmer
from nltk import word_tokenize, pos_tag
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


pd.set_option('display.max_columns', None)

from gensim.models import Word2Vec


from matplotlib import pyplot as plt
import seaborn as sns


from plotly.subplots import make_subplots
import plotly.graph_objects as go

#Use fivethirtyeight style
plt.style.use('fivethirtyeight')

#Cancel any style and reset matplotlib styles
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]


In [4]:
df = pd.read_csv("C:/Users/dimit/OneDrive/Desktop/git_projects/Recommendation System/Recommendation-System/n_movies.csv")
df

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413"
...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,After an experimental gene therapy turns them ...,"['Morgan Taylor Campbell, ', 'Italia Ricci, ',...","3,130"
9953,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a com...,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melis...","970,067"
9954,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,898"
9955,Supernatural,(2005–2020),TV-14,44 min,"Drama, Fantasy, Horror",8.4,Two brothers follow their father's footsteps a...,"['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim ...","439,601"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9957 entries, 0 to 9956
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        9957 non-null   object 
 1   year         9430 non-null   object 
 2   certificate  6504 non-null   object 
 3   duration     7921 non-null   object 
 4   genre        9884 non-null   object 
 5   rating       8784 non-null   float64
 6   description  9957 non-null   object 
 7   stars        9957 non-null   object 
 8   votes        8784 non-null   object 
dtypes: float64(1), object(8)
memory usage: 700.2+ KB


In [6]:
df.drop_duplicates(subset=['title'], keep='first', inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7912 entries, 0 to 9912
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        7912 non-null   object 
 1   year         7410 non-null   object 
 2   certificate  4737 non-null   object 
 3   duration     6459 non-null   object 
 4   genre        7848 non-null   object 
 5   rating       6872 non-null   float64
 6   description  7912 non-null   object 
 7   stars        7912 non-null   object 
 8   votes        6872 non-null   object 
dtypes: float64(1), object(8)
memory usage: 618.1+ KB


In [8]:
df['title'] = df['title'].str.replace(' ', '_')

In [9]:
df.duration = df.duration.str.replace('min', '')
df.duration.fillna(0, inplace=True) 
df.duration = df.duration.astype(int)
df.duration.replace(0, df.duration.mean(), inplace=True)

In [10]:
genres = df['genre'].str.split(', ', expand=True)
df = pd.concat([df, genres], axis=1)

In [11]:
def clean_stars(stars_string):
    cleaned_string = stars_string.strip("[]").replace("'", "").replace(", ", ",").split(",")
    return list(map(str.strip, cleaned_string))

# Apply the cleaning function to the stars column
df['stars'] = df['stars'].map(clean_stars)


def clean_stars(stars_list):
    return [name for name in stars_list if name]  # Keep only non-empty strings

# Apply the cleaning function to the stars column
df['stars'] = df['stars'].apply(clean_stars)

In [12]:
stars_expanded = df['stars'].apply(pd.Series)

C:\Users\dimit\AppData\Local\Temp\ipykernel_11148\3418863794.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stars_expanded = df['stars'].apply(pd.Series)


In [13]:
stars_expanded = stars_expanded.iloc[:, :3]
stars_expanded.rename(columns={0: 'Alpha', 1: 'Beta', 2: 'Gamma'}, inplace=True)

In [14]:
df = pd.concat([df, stars_expanded], axis=1)

In [15]:
words = stopwords.words("english")

In [16]:
lemmatizer = WordNetLemmatizer()

In [17]:
df['description'] = df['description'].apply(lambda x: " ".join([lemmatizer.lemmatize(i) for i in re.sub("[^a-zA-Z]", " ", str(x)).split() if i not in words]).lower())

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dimit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dimit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [20]:
df['tokenized'] = df['description'].apply(word_tokenize)

In [21]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\dimit\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [22]:
df['POS'] = df['tokenized'].apply(pos_tag)

In [23]:
# tag_filter = ["NN","NNS","NNP","NNPS"]  #keep only nouns, adjectives and adverbs
tag_filter = ["NN","NNS","NNP","NNPS","RB","RBR","RBS","JJ","JJR","JJS"] 
# filter pos function
def pos_filter(row):
    keep_list = []
    for x,y in row:
        if y in tag_filter:
            keep_list.append(x)
    return keep_list
df['clean'] = df['POS'].apply(pos_filter)

df.head()

,title,year,certificate,duration,genre,rating,description,stars,votes,0,1,2,Alpha,Beta,Gamma,tokenized,POS,clean
0,Cobra_Kai,(2018– ),TV-14,30.0,"Action, Comedy, Drama",8.5,decades all valley karate tournament bout midd...,"[Ralph Macchio, William Zabka, Courtney Hengge...","177,031",Action,Comedy,Drama,Ralph Macchio,William Zabka,Courtney Henggeler,"[decades, all, valley, karate, tournament, bou...","[(decades, NNS), (all, DT), (valley, VBP), (ka...","[decades, tournament, middle, daniel, larusso,..."
1,The_Crown,(2016– ),TV-MA,58.0,"Biography, Drama, History",8.7,follows political rivalry romance queen elizab...,"[Claire Foy, Olivia Colman, Imelda Staunton, M...","199,885",Biography,Drama,History,Claire Foy,Olivia Colman,Imelda Staunton,"[follows, political, rivalry, romance, queen, ...","[(follows, VBZ), (political, JJ), (rivalry, NN...","[political, rivalry, romance, queen, ii, reign..."
2,Better_Call_Saul,(2015–2022),TV-MA,46.0,"Crime, Drama",8.9,the trial tribulation criminal lawyer jimmy mc...,"[Bob Odenkirk, Rhea Seehorn, Jonathan Banks, P...","501,384",Crime,Drama,None,Bob Odenkirk,Rhea Seehorn,Jonathan Banks,"[the, trial, tribulation, criminal, lawyer, ji...","[(the, DT), (trial, NN), (tribulation, NN), (c...","[trial, tribulation, criminal, lawyer, jimmy, ..."
3,Devil_in_Ohio,(2022),TV-MA,356.0,"Drama, Horror, Mystery",5.9,when psychiatrist shelter mysterious cult esca...,"[Emily Deschanel, Sam Jaeger, Gerardo Celasco,...","9,773",Drama,Horror,Mystery,Emily Deschanel,Sam Jaeger,Gerardo Celasco,"[when, psychiatrist, shelter, mysterious, cult...","[(when, WRB), (psychiatrist, NN), (shelter, RB...","[psychiatrist, shelter, mysterious, cult, esca..."
4,Cyberpunk:_Edgerunners,(2022– ),TV-MA,24.0,"Animation, Action, Adventure",8.6,a street kid trying survive technology body mo...,"[Zach Aguilar, Kenichiro Ohashi, Emi Lo, Aoi Y...","15,413",Animation,Action,Adventure,Zach Aguilar,Kenichiro Ohashi,Emi Lo,"[a, street, kid, trying, survive, technology, ...","[(a, DT), (street, NN), (kid, NN), (trying, VB...","[street, kid, survive, technology, body, modif..."


In [24]:
def put_title_first(row):
    title = row['title']
    description = row['clean']
    if isinstance(description, list):
        description.insert(0, title)
    else:
        description = [title, description]
    return description


def put_genre1(row):
    genre1 = row[0]
    description = row['clean']
    if isinstance(description, list):
        description.insert(1, genre1)
    else:
        description = [genre1, description]
    return description

def put_genre2(row):
    genre2 = row[1]
    description = row['clean']
    if isinstance(description, list):
        description.insert(2, genre2)
    else:
        description = [genre2, description]
    return description

def put_genre3(row):
    genre3 = row[2]
    description = row['clean']
    if isinstance(description, list):
        description.insert(3, genre3)
    else:
        description = [genre3, description]
    return description


def put_stars(row):
    names = row['Alpha']
    description = row['clean']
    if isinstance(description, list):
        description.insert(4, names)
    else:
        description = [names, description]
    return description

def put_co_star(row):
    names = row['Beta']
    description = row['clean']
    if isinstance(description, list):
        description.insert(5, names)
    else:
        description = [names, description]
    return description

In [25]:
df['movie_description'] = df.apply(put_title_first, axis=1)
df['movie_description'] = df.apply(put_genre1, axis=1)
# df['movie_description'] = df.apply(put_genre2, axis=1)
# df['movie_description'] = df.apply(put_genre3, axis=1)
df['movie_description'] = df.apply(put_stars, axis=1)
df['movie_description'] = df.apply(put_co_star, axis=1)

In [26]:
df.reset_index(drop=True, inplace=True)


In [27]:
df

,title,year,certificate,duration,genre,rating,description,stars,votes,0,1,2,Alpha,Beta,Gamma,tokenized,POS,clean,movie_description
0,Cobra_Kai,(2018– ),TV-14,30.000000,"Action, Comedy, Drama",8.5,decades all valley karate tournament bout midd...,"[Ralph Macchio, William Zabka, Courtney Hengge...","177,031",Action,Comedy,Drama,Ralph Macchio,William Zabka,Courtney Henggeler,"[decades, all, valley, karate, tournament, bou...","[(decades, NNS), (all, DT), (valley, VBP), (ka...","[Cobra_Kai, Action, decades, tournament, Ralph...","[Cobra_Kai, Action, decades, tournament, Ralph..."
1,The_Crown,(2016– ),TV-MA,58.000000,"Biography, Drama, History",8.7,follows political rivalry romance queen elizab...,"[Claire Foy, Olivia Colman, Imelda Staunton, M...","199,885",Biography,Drama,History,Claire Foy,Olivia Colman,Imelda Staunton,"[follows, political, rivalry, romance, queen, ...","[(follows, VBZ), (political, JJ), (rivalry, NN...","[The_Crown, Biography, political, rivalry, Cla...","[The_Crown, Biography, political, rivalry, Cla..."
2,Better_Call_Saul,(2015–2022),TV-MA,46.000000,"Crime, Drama",8.9,the trial tribulation criminal lawyer jimmy mc...,"[Bob Odenkirk, Rhea Seehorn, Jonathan Banks, P...","501,384",Crime,Drama,None,Bob Odenkirk,Rhea Seehorn,Jonathan Banks,"[the, trial, tribulation, criminal, lawyer, ji...","[(the, DT), (trial, NN), (tribulation, NN), (c...","[Better_Call_Saul, Crime, trial, tribulation, ...","[Better_Call_Saul, Crime, trial, tribulation, ..."
3,Devil_in_Ohio,(2022),TV-MA,356.000000,"Drama, Horror, Mystery",5.9,when psychiatrist shelter mysterious cult esca...,"[Emily Deschanel, Sam Jaeger, Gerardo Celasco,...","9,773",Drama,Horror,Mystery,Emily Deschanel,Sam Jaeger,Gerardo Celasco,"[when, psychiatrist, shelter, mysterious, cult...","[(when, WRB), (psychiatrist, NN), (shelter, RB...","[Devil_in_Ohio, Drama, psychiatrist, shelter, ...","[Devil_in_Ohio, Drama, psychiatrist, shelter, ..."
4,Cyberpunk:_Edgerunners,(2022– ),TV-MA,24.000000,"Animation, Action, Adventure",8.6,a street kid trying survive technology body mo...,"[Zach Aguilar, Kenichiro Ohashi, Emi Lo, Aoi Y...","15,413",Animation,Action,Adventure,Zach Aguilar,Kenichiro Ohashi,Emi Lo,"[a, street, kid, trying, survive, technology, ...","[(a, DT), (street, NN), (kid, NN), (trying, VB...","[Cyberpunk:_Edgerunners, Animation, street, ki...","[Cyberpunk:_Edgerunners, Animation, street, ki..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7907,Alex,(2017–2019),NaN,47.000000,"Action, Crime, Thriller",7.3,add plot,"[Alain Darborg, |, Stars:, Dragomir Mrsic, Rak...",33,Action,Crime,Thriller,Alain Darborg,|,Stars:,"[add, plot]","[(add, NN), (plot, NN)]","[Alex, Action, add, plot, Alain Darborg, |]","[Alex, Action, add, plot, Alain Darborg, |]"
7908,The_Weekly_with_Wendy_Mesley,(2018– ),NaN,65.632204,"News, Reality-TV",NaN,how ontario popular new premier get unpopular ...,"[Wendy Mesley, Billy Porter, Jacob Tobia, Mark...",NaN,News,Reality-TV,None,Wendy Mesley,Billy Porter,Jacob Tobia,"[how, ontario, popular, new, premier, get, unp...","[(how, WRB), (ontario, JJ), (popular, JJ), (ne...","[The_Weekly_with_Wendy_Mesley, News, ontario, ...","[The_Weekly_with_Wendy_Mesley, News, ontario, ..."
7909,The_Drew_Barrymore_Show,(2020– ),TV-PG,44.000000,Talk-Show,6.0,in inaugural episode the drew barrymore show d...,"[Adam Heydt, |, Stars:, Drew Barrymore, Camero...",19,Talk-Show,None,None,Adam Heydt,|,Stars:,"[in, inaugural, episode, the, drew, barrymore,...","[(in, IN), (inaugural, JJ), (episode, NN), (th...","[The_Drew_Barrymore_Show, Talk-Show, inaugural...","[The_Drew_Barrymore_Show, Talk-Show, inaugural..."
7910,Hollywood_Insider,(2018– ),NaN,65.632204,Talk-Show,NaN,behind scene the irishman,"[Bobby Cannavale, Robert De Niro, Al Pacino, J...",NaN,Talk-Show,None,None,Bobby Cannavale,Robert De Niro,Al Pacino,"[behind, scene, the, irishman]","[(behind, IN), (scene, PDT), (the, DT), (irish...","[Hollywood_Insider, Talk-Show, irishman, Bobby..."

In [28]:
genres_no_movies_no_series = ['Game-Show', 'Reality-TV', 'News', 'Talk-Show']

# Create a new DataFrame containing only the rows with the specified genres
reality = df[df['genre'].isin(genres_no_movies_no_series)]

In [29]:
df = df[~df['genre'].isin(genres_no_movies_no_series)]

In [30]:
df['year'] = df['year'].fillna('')
series = df[df['year'].str.contains('–')]
movies = df[~df['year'].str.contains('–')]

C:\Users\dimit\AppData\Local\Temp\ipykernel_11148\1300221284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['year'].fillna('')


In [31]:
list_of_movies = movies['movie_description'].tolist()
list_of_series = series['movie_description'].tolist()

In [32]:
model_movies = Word2Vec(list_of_movies, vector_size=10, window=10, min_count=1, workers=4, sample=1e-3, sg=1, negative=5)


In [33]:
model_series = Word2Vec(list_of_series, vector_size=30, window=8, min_count=1, workers=4, sample=1e-3, sg=1, negative=5)

In [38]:
# similarity_dict = {}  # Create an empty dictionary to store the similarities

# titles = movies.title.tolist()  # Convert the titles column to a list

# # Iterate through all pairs of titles
# for i in range(len(titles)):
#     for j in range(len(titles)):
#         title1 = titles[i]
#         title2 = titles[j]
        
#         # Skip if title1 and title2 are the same
#         if title1 != title2:
#             # Check if both titles exist in the model's vocabulary
#             if title1 in model_movies.wv and title2 in model_movies.wv:
#                 similarity = model_movies.wv.similarity(title1, title2)
                
#                 # Store in dictionary and print only if similarity is greater than 0.8
#                 if similarity > 0.96:
#                     similarity_dict[(title1, title2)] = similarity
#                     print(f"Similarity between '{title1}' and '{title2}': {similarity}")

titles = movies.title.tolist()

for i in range(len(titles)):
    similarity = model_movies.wv.similarity('Bird_Box', titles[i])
    if similarity > 0.9:
        print(f"Bird_Box' and '{titles[i]}': {similarity}")


               
       
               


Bird_Box' and 'The_Haunting_of_Hill_House': 0.9034386873245239
Bird_Box' and '2001:_A_Space_Odyssey': 0.9223955273628235
Bird_Box' and 'Uncut_Gems': 0.918688178062439
Bird_Box' and 'Below_Her_Mouth': 0.9224051833152771
Bird_Box' and 'The_Human_Centipede_(First_Sequence)': 0.9055659174919128
Bird_Box' and 'tick,_tick..._BOOM!': 0.9008433818817139
Bird_Box' and 'Bird_Box': 1.0
Bird_Box' and 'Contagion': 0.977662205696106
Bird_Box' and 'One_Piece': 0.9085851907730103
Bird_Box' and 'Brazen': 0.9397001266479492
Bird_Box' and 'The_Society': 0.9449598789215088
Bird_Box' and 'Bill_&_Ted_Face_the_Music': 0.9516276121139526
Bird_Box' and 'Clickbait': 0.9734592437744141
Bird_Box' and 'The_Innocent': 0.9183682203292847
Bird_Box' and 'Rise_of_the_Teenage_Mutant_Ninja_Turtles:_The_Movie': 0.9245063066482544
Bird_Box' and 'This_Means_War': 0.9091538190841675
Bird_Box' and 'Smokin'_Aces': 0.9513452053070068
Bird_Box' and 'The_King:_Eternal_Monarch': 0.9150716662406921
Bird_Box' and 'Wish_Dragon': 0.90